**Imports**

PyTorch Geometric

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 13.8 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 14.9 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.5 MB 15.3 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 750 kB 14.7 MB/s 
     |████████████████████████████████| 407 kB 16.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=0015ff2f2af58feb18fc6356044573c1312758314186109955a351d1d8c86e6c
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
import pandas as pd
import numpy as np

from torch_geometric.data import InMemoryDataset, download_url
from datetime import datetime

**Mount GDrive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
action_types.csv              participants_activity_anomalies.csv
activities.csv                patients.csv
activity_types.csv            scores.csv
anomalies.csv                 scores_predictions.csv
anomaly_types.csv             sensor_locations.csv
diagnosis_predictions.csv     sensors.csv
diagnosis_types.csv           tasks.csv
events.csv                    task_types.csv
overt_error_action_types.csv  trajectory_anomalies.csv


In [ ]:
'''
TODO:
  - cercare come ocnvertire il tipo della colonna del dataframe da stringa a data
'''
def task_activity_sensors(tasks, events, sensors, activities, patient):

  # Seleziona i task-activity eseguiti dal paziente 'patient'
  task_activity = tasks.query('patient == ' + patient)

  # Seleziona il tempo di inizio e fine delle attività svolte dal paziente 'patient'
  activities_patient = activities.query('patient == ' + patient)
  start_time = activities_patient.iloc[0]['start']
  end_time = activities_patient.iloc[len(activities_patient) - 1]['end']

  # Query nel dataframe events per selezionare solo i sensori utilizzati nelle attività
  sensors_events = events.query('patient == ' + patient + ' and time >= "' + start_time + '" and time <= "' + end_time + '"')

  # Matrice risultato che contiene i dati dei nodi
  res_node = pd.DataFrame(columns=['features'])
  res_edge = pd.DataFrame(columns=['times'])

  for i in range(0, len(task_activity)):
    pair_str = task_activity.iloc[i]['task'] + '-' + task_activity.iloc[i]['activity']
    s_time = task_activity.iloc[i]['time']
    e_time = task_activity.iloc[i + 1]['time']

    if i != len(task_activity) - 1:
      if pair_str in res_edge.index:
        res_edge.at[pair_str, 'times'] = res_edge.loc[pair_str, 'times'].append((s_time, e_time))
      
      else:
        res_edge.loc[pair_str] = [[(s_time, e_time)]]

    if pair_str in res_node.index

In [ ]:
'''
TODO:
  - Controllare le righe della stessa coppia task-activity per inserire un unico blocco temporale (riga 28)
  - Inserire le features del nodo (riga 34)
'''

In [ ]:
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
tasks['time'] = pd.to_datetime(tasks['time'], infer_datetime_format=True)
#tasks['time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S.%f'))

In [ ]:
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
res = tasks.query('patient == 88')

In [ ]:
res

,patient,time,task,activity
0,88,09:21:17.2387,1,1
1,88,09:21:17.2387,3,1
2,88,09:21:28.37237,4,1
3,88,09:21:29.83978,4,1
4,88,09:21:32.87967,4,1
...,...,...,...,...
134,88,10:27:14.50977,5,15
135,88,10:27:58.60021,1,16
136,88,10:27:59.22573,2,16
137,88,10:28:07.03055,2,16


In [ ]:
print(tasks.iloc[0]['time'])

09:21:17.2387


In [ ]:
print(datetime.strftime(tasks.iloc[0]['time'], '%H:%M:%S:%f'))

09:21:17:238700


In [ ]:
datetime.strftime(tasks.iloc[0]['time'], '%H:%M:%S:%f') == datetime.strftime(tasks.iloc[1]['time'], '%H:%M:%S:%f')

True

In [ ]:
type(tasks.iloc[0]['time'])

str

In [ ]:
res

,patient,time,task,activity
0,88,2022-04-13 09:21:17.238700,1,1
1,88,2022-04-13 09:21:17.238700,3,1
2,88,2022-04-13 09:21:28.372370,4,1
3,88,2022-04-13 09:21:29.839780,4,1
4,88,2022-04-13 09:21:32.879670,4,1
...,...,...,...,...
134,88,2022-04-13 10:27:14.509770,5,15
135,88,2022-04-13 10:27:58.600210,1,16
136,88,2022-04-13 10:27:59.225730,2,16
137,88,2022-04-13 10:28:07.030550,2,16


In [ ]:
a = res['time'].apply(lambda x: datetime.strftime(x, '%H:%M:%S.%f'))

In [ ]:
type(a.iloc[0])

str

In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')

,patient,time,task,activity
0,88,09:21:17.2387,1,1
1,88,09:21:17.2387,3,1
2,88,09:21:28.37237,4,1
3,88,09:21:29.83978,4,1
4,88,09:21:32.87967,4,1
...,...,...,...,...
69209,329,15:09:03.07727,1,15
69210,329,15:09:03.70765,2,15
69211,329,15:09:14.67948,2,15
69212,329,15:09:18.98124,3,15


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/events.csv')

,patient,time,sensor,value
0,88,09:00:05.78693,m010,on
1,88,09:00:08.08177,m010,off
2,88,09:00:13.71834,m013,off
3,88,09:00:19.76291,m010,on
4,88,09:00:21.6229,m013,on
...,...,...,...,...
1209934,329,16:20:29.92485,m022,off
1209935,329,16:20:32.888,m021,off
1209936,329,16:20:32.91215,m025,off
1209937,329,16:20:33.9529,d001,close


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

,patient,activity_type,start,end
0,88,1,09:21:17.2387,09:24:10.78761
1,88,2,09:25:15.5674,09:28:42.21513
2,88,3,09:30:21.23501,09:33:50.33897
3,88,4,09:34:53.77164,09:42:28.82678
4,88,5,09:43:36.54254,09:44:33.83539
...,...,...,...,...
5274,329,11,14:36:47.11877,14:40:02.51782
5275,329,12,14:44:22.30159,14:47:07.77214
5276,329,13,14:48:19.93743,14:52:49.75592
5277,329,14,14:53:32.62822,15:02:49.75547


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/task_types.csv')

,activity_id,task_id,description,action_type
0,1,1,Participant retrieves broom from supply closet,1
1,1,2,Participant retrieves duster from supply closet,1
2,1,3,Participant retrieves dust pan and brush from ...,1
3,1,4,Participant sweeps kitchen floor,5
4,1,5,Participant uses dust pan and brush,5
...,...,...,...,...
104,16,3,Participant brings deck of cards to the dining...,5
105,16,4,Participant sets up card game,5
106,16,5,Participant answers the telephone,11
107,16,6,Participant explains the rules of the game ove...,5


In [ ]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to `self.raw_dir`.
        download_url(url, self.raw_dir)
        ...

    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])